In [1]:
import os

In [2]:
%pwd

'd:\\MLOPS\\ML-Approach-for-Predict-Cancellation-Prevent-Loss-with-MLflow\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'd:\\MLOPS\\ML-Approach-for-Predict-Cancellation-Prevent-Loss-with-MLflow'

In [7]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/pritamnarwade11/Predicting-Cancellations-to-Prevent-Revenue-Loss-with-MLflow.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="pritamnarwade11"
os.environ["MLFLOW_TRACKING_PASSWORD"]="871b76c6a3213893886036cf5681b2454522ec97"  

In [18]:
from dataclasses import dataclass
from pathlib import Path 

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path 
    test_data_path: Path 
    model_path: Path 
    all_params: dict 
    metric_file_name: Path 
    target_column: str 
    mlflow_uri: str 

In [21]:
from src.constants import *
from src.utils.common import *

In [19]:
class ConfigurationManager:
    def __init__(self, 
                 config_filepath = CONFIG_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH ):
        
        self.config = read_yaml(config_filepath)
        self.schema = read_yaml(schema_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        schema = self.schema.TARGET_COLUMN
        params = self.params.RandomForestClassifier

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path= config.test_data_path,
            model_path= config.model_path,
            all_params= params,
            metric_file_name= config.metric_file_name,
            target_column= schema.name, 
            mlflow_uri= "https://dagshub.com/pritamnarwade11/Predicting-Cancellations-to-Prevent-Revenue-Loss-with-MLflow.mlflow"
        )


        return model_evaluation_config 

In [20]:
import os 
import pandas as pd 
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score 
from urllib.parse import urlparse 
import mlflow 
import mlflow.sklearn 
import joblib

In [16]:
class ModelEvaluation:
    def __init__(self, config=ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred) 
        r2 = r2_score(actual, pred) 
        return rmse, mae, r2 
    
    def log_into_mlflow(self):
        print('Engineering features...')
        test_data = np.load(self.config.test_data_path)

        print('Loading model...')
        model = joblib.load(self.config.model_path) 

        X_test = test_data[:,:-1]
        y_test = test_data[:, -1]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        # mlflow
        with mlflow.start_run() as run:
            y_pred = model.predict(X_test)

            (rmse, mae, r2) = self.eval_metrics(y_test, y_pred)

            # savings metrics as local 
            scores = {"rmse": rmse, "mae": mae, "r2":r2}
            save_json(path_tosave=Path(self.config.metric_file_name), data=scores)

            # Log parameters and metrics using the MLflow APIs
            mlflow.log_params(self.config.all_params)


            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("mae", mae)
            mlflow.log_metric("r2", r2)

                # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                # https://mlflow.org/docs/1.24.0/tutorials-and-examples/tutorial.html
                mlflow.sklearn.log_model(model, "model", registered_model_name="RandomForestClassifier")
            else:
                mlflow.sklearn.log_model(model, "model")



In [22]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise CustomException(e,sys)

[ 2024-04-01 17:23:29,079 ] 31 root - INFO - config\config.yaml yaml file loaded successfully
[ 2024-04-01 17:23:29,095 ] 31 root - INFO - schema.yaml yaml file loaded successfully
[ 2024-04-01 17:23:29,101 ] 31 root - INFO - params.yaml yaml file loaded successfully
[ 2024-04-01 17:23:29,107 ] 52 root - INFO - Created directory at : artifacts
[ 2024-04-01 17:23:29,111 ] 52 root - INFO - Created directory at : artifacts/model_evaluation


Engineering features...
Loading model...
[ 2024-04-01 17:23:30,748 ] 67 root - INFO - JSON file saved at artifacts\model_evaluation\metrics.json


Registered model 'RandomForestClassifier' already exists. Creating a new version of this model...
2024/04/01 17:24:21 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: RandomForestClassifier, version 2
Created version '2' of model 'RandomForestClassifier'.
